In [0]:
#@title Download Data 

!wget !wget https://www.dropbox.com/s/bipavlbb8v7q5ai/corpus.zip 
!unzip corpus.zip
!wget https://www.dropbox.com/s/y74zmnvbg66awba/test_small.zip
!unzip test_small.zip

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: Images/THEODORE_GERICAULT_30.jpg  
  inflating: Images/THEODORE_GERICAULT_31.jpg  
  inflating: Images/THEODORE_GERICAULT_32.jpg  
  inflating: Images/THEODORE_GERICAULT_33.jpg  
  inflating: Images/THEODORE_GERICAULT_34.jpg  
  inflating: Images/THEODORE_GERICAULT_35.jpg  
  inflating: Images/THEODORE_GERICAULT_36.jpg  
  inflating: Images/THEODORE_GERICAULT_37.jpg  
  inflating: Images/THEODORE_GERICAULT_38.jpg  
  inflating: Images/THEODORE_GERICAULT_39.jpg  
  inflating: Images/THEODORE_GERICAULT_4.jpg  
  inflating: Images/THEODORE_GERICAULT_40.jpg  
  inflating: Images/THEODORE_GERICAULT_41.jpg  
  inflating: Images/THEODORE_GERICAULT_42.jpg  
  inflating: Images/THEODORE_GERICAULT_43.jpg  
  inflating: Images/THEODORE_GERICAULT_44.jpg  
  inflating: Images/THEODORE_GERICAULT_45.jpg  
  inflating: Images/THEODORE_GERICAULT_46.jpg  
  inflating: Images/THEODORE_GERICAULT_47.jpg  
  inflating: Images/THEODORE

In [0]:
import pandas as pd
from keras_preprocessing.image import ImageDataGenerator

In [0]:
trainAuthors = pd.read_csv("/content/trainAuthor.csv",dtype=str)
testAuthors = pd.read_csv("/content/testAuthor.csv",dtype=str)

Load a train (80%) / test (20%) split 

All the classes are balanced

In [0]:
trainAuthors.groupby("author").count()

,Unnamed: 0,fileName
author,,
0,34,34
1,40,40
10,40,40
11,40,40
12,31,31
...,...,...
87,34,34
88,36,36
89,31,31


In [0]:
testAuthors.groupby("author").count()

,Unnamed: 0,fileName
author,,
0,9,9
1,10,10
10,10,10
11,10,10
12,8,8
...,...,...
87,9,9
88,9,9
89,8,8


Load images from dataframe and apply data augmentation [Keras image preprocessing](https://keras.io/preprocessing/image/)

In [0]:

train_datagen = ImageDataGenerator(
        rescale=1./255,
        width_shift_range=0.05,
        height_shift_range=0.05,
        rotation_range=5,
        zoom_range=[1.0,1.05],
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
        dataframe=trainAuthors,
        directory='/content/Images/',
        x_col="fileName",
        y_col="author",
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_dataframe(
        dataframe=testAuthors,
        directory='/content/Images/',
        x_col="fileName",
        y_col="author",
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')



Found 3412 validated image filenames belonging to 91 classes.
Found 854 validated image filenames belonging to 91 classes.


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization as BN
from keras.layers import GaussianNoise as GN
from keras.optimizers import SGD

from keras.applications.vgg16 import VGG16
#from keras.layers import Dense, Dropout, Activation, Flatten, Lambda, Input
from keras.optimizers import SGD, Adam
from keras.models import Model
from keras.callbacks import LearningRateScheduler as LRS


Using TensorFlow backend.


In [0]:
num_clases = 90
## DEF A BLOCK CONV + BN + GN + MAXPOOL
def CBGN(model,filters,ishape=0):
    if (ishape!=0):
        model.add(Conv2D(filters, (3, 3), padding='same',input_shape=ishape))
    else:
        model.add(Conv2D(filters, (3, 3), padding='same'))
        
    model.add(BN())
    model.add(GN(0.15))
    model.add(Activation('relu'))
    
    model.add(Conv2D(filters, (3, 3), padding='same'))    
    model.add(BN())
    model.add(GN(0.15))
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))
  
    return model

## DEF NN TOPOLOGY  

for layer in modelVGG.layers:
  layer.trainable  = False
conv=modelVGG.get_layer('fc1').output 
#out = Flatten()(conv)
predictions=Dense(num_clases, activation='softmax', name='predictions')(conv)
model = Model(inputs=modelVGG.input, outputs=predictions)
print(model.summary())






553467904/553467096 [==============================] - 6s 0us/step






Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
________________________________________________

In [0]:
opt = Adam(lr=0.0001, decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
def scheduler(epoch):
    if epoch < 50:
        return .0001
    elif epoch < 100:
      return 0.00001
    else:
      return 0.000001
set_lr = LRS(scheduler)

In [0]:
model.fit_generator(
        train_generator,
        steps_per_epoch=200,
        epochs=250,
        callbacks=[set_lr],
        validation_data=validation_generator)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/250
200/200 [==============================] - 82s 409ms/step - loss: 4.4899 - acc: 0.0380 - val_loss: 4.0899 - val_acc: 0.0923
Epoch 2/250
200/200 [==============================] - 77s 384ms/step - loss: 3.8752 - acc: 0.1192 - val_loss: 3.7195 - val_acc: 0.1420
Epoch 3/250
200/200 [==============================] - 77s 386ms/step - loss: 3.5419 - acc: 0.1745 - val_loss: 3.4632 - val_acc: 0.1870
Epoch 4/250
200/200 [==============================] - 77s 385ms/step - loss: 3.3070 - acc: 0.2181 - val_loss: 3.3203 - val_acc: 0.2213
Epoch 5/250
200/200 [==============================] - 77s 385ms/step - loss: 3.1208 - acc: 0.2572 - val_loss: 3.1768 - val_acc: 0.2296
Epoch 6/250
200/200 [==============================] - 78s 388ms/step - loss: 2.9912 - acc: 0.2922 - val_loss: 3.0660 - val_acc: 0.2592
Epoch 7/250
200/200 [==============================] - 76s 382ms/step - loss: 2.8498 - a

KeyboardInterrupt: ignored

In [0]:
def scheduler(epoch):
    if epoch < 50:
        return .000001
    elif epoch < 100:
      return 0.000001
    else:
      return 0.0000001
set_lr = LRS(scheduler)
model.fit_generator(
        train_generator,
        steps_per_epoch=200,
        epochs=250,
        callbacks=[set_lr],
        validation_data=validation_generator)

Epoch 1/250
200/200 [==============================] - 76s 382ms/step - loss: 1.3168 - acc: 0.6981 - val_loss: 2.0866 - val_acc: 0.4746
Epoch 2/250
200/200 [==============================] - 75s 374ms/step - loss: 1.2795 - acc: 0.7089 - val_loss: 2.0570 - val_acc: 0.4970
Epoch 3/250
200/200 [==============================] - 75s 374ms/step - loss: 1.2476 - acc: 0.7241 - val_loss: 2.0463 - val_acc: 0.5077
Epoch 4/250
200/200 [==============================] - 75s 375ms/step - loss: 1.2362 - acc: 0.7311 - val_loss: 2.0426 - val_acc: 0.5136
Epoch 5/250
200/200 [==============================] - 75s 376ms/step - loss: 1.2491 - acc: 0.7263 - val_loss: 2.0425 - val_acc: 0.5136
Epoch 6/250
200/200 [==============================] - 75s 373ms/step - loss: 1.2316 - acc: 0.7300 - val_loss: 2.0420 - val_acc: 0.5136
Epoch 7/250
200/200 [==============================] - 74s 369ms/step - loss: 1.2401 - acc: 0.7292 - val_loss: 2.0424 - val_acc: 0.5136
Epoch 8/250
200/200 [===========================

In [0]:
modelVGG.summary()

In [0]:
model.summary()